In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Hourly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 414  21

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [6]:
dlist= load('Hourly_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 414   5   4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
  0   1   2   3   4 
 60  24  47  87 196 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

2
1
0
2
2
3
2
3
2
0
3


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [35]:
sum(train_data$length!=700)

[1] 174

In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:2.055552 
[2]	train-rmse:1.581338 
[3]	train-rmse:1.241363 
[4]	train-rmse:0.948505 
[5]	train-rmse:0.780854 
[6]	train-rmse:0.634040 
[7]	train-rmse:0.528883 
[8]	train-rmse:0.453003 
[9]	train-rmse:0.390535 
[10]	train-rmse:0.342635 
[11]	train-rmse:0.318436 
[12]	train-rmse:0.293496 
[13]	train-rmse:0.271975 
[14]	train-rmse:0.241538 
[15]	train-rmse:0.217078 
[16]	train-rmse:0.198611 
[17]	train-rmse:0.192890 
[18]	train-rmse:0.179187 
[19]	train-rmse:0.168761 
[20]	train-rmse:0.157763 
[21]	train-rmse:0.145854 
[22]	train-rmse:0.138970 
[23]	train-rmse:0.132660 
[24]	train-rmse:0.121680 
[25]	train-rmse:0.115258 
[26]	train-rmse:0.108415 
[27]	train-rmse:0.101210 
[28]	train-rmse:0.094298 
[29]	train-rmse:0.089564 
[30]	train-rmse:0.083893 
[31]	train-rmse:0.079640 
[32]	train-rmse:0.075931 
[33]	train-rmse:0.072580 
[34]	train-rmse:0.069053 
[35]	train-rmse:0.065047 
[36]	train-rmse:0.060361 
[37]	train-rmse:0.059163 
[38]	train-rmse:0.055006 
[39]	train-rmse:0.052

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.291170 
[2]	train-mlogloss:1.053770 
[3]	train-mlogloss:0.884216 
[4]	train-mlogloss:0.744844 
[5]	train-mlogloss:0.642600 
[6]	train-mlogloss:0.561388 
[7]	train-mlogloss:0.489233 
[8]	train-mlogloss:0.423887 
[9]	train-mlogloss:0.375083 
[10]	train-mlogloss:0.336223 
[11]	train-mlogloss:0.302494 
[12]	train-mlogloss:0.271111 
[13]	train-mlogloss:0.250942 
[14]	train-mlogloss:0.232611 
[15]	train-mlogloss:0.209946 
[16]	train-mlogloss:0.189829 
[17]	train-mlogloss:0.176316 
[18]	train-mlogloss:0.164137 
[19]	train-mlogloss:0.149173 
[20]	train-mlogloss:0.135705 
[21]	train-mlogloss:0.125065 
[22]	train-mlogloss:0.116029 
[23]	train-mlogloss:0.107451 
[24]	train-mlogloss:0.100427 
[25]	train-mlogloss:0.094385 
[26]	train-mlogloss:0.089990 
[27]	train-mlogloss:0.085774 
[28]	train-mlogloss:0.080729 
[29]	train-mlogloss:0.077212 
[30]	train-mlogloss:0.073250 
[31]	train-mlogloss:0.069776 
[32]	train-mlogloss:0.067349 
[33]	train-mlogloss:0.064301 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1809
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 21
[LightGBM] [Info] Start training from score 2.812287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026404 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1809
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 21
[LightGBM] [Info] Start training from score -1.918972
[LightGBM] [Info] Start training from score -2.735734
[LightGBM] [Info] Start training from score -2.214437
[LightGBM] [Info] Start training from score -1.672839
[LightGBM] [Info] Start training from score -0.710359
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [23]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [24]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.548617 
[2]	train-rmse:1.228104 
[3]	train-rmse:1.006942 
[4]	train-rmse:0.853554 
[5]	train-rmse:0.716040 
[6]	train-rmse:0.590094 
[7]	train-rmse:0.502666 
[8]	train-rmse:0.460181 
[9]	train-rmse:0.425960 
[10]	train-rmse:0.385761 
[11]	train-rmse:0.347817 
[12]	train-rmse:0.316093 
[13]	train-rmse:0.276653 
[14]	train-rmse:0.263653 
[15]	train-rmse:0.250201 
[16]	train-rmse:0.239457 
[17]	train-rmse:0.215273 
[18]	train-rmse:0.198574 
[19]	train-rmse:0.179061 
[20]	train-rmse:0.166620 
[21]	train-rmse:0.157180 
[22]	train-rmse:0.146059 
[23]	train-rmse:0.136852 
[24]	train-rmse:0.130934 
[25]	train-rmse:0.121659 
[26]	train-rmse:0.116327 
[27]	train-rmse:0.111112 
[28]	train-rmse:0.107177 
[29]	train-rmse:0.100491 
[30]	train-rmse:0.097034 
[31]	train-rmse:0.092890 
[32]	train-rmse:0.083602 
[33]	train-rmse:0.080786 
[34]	train-rmse:0.078535 
[35]	train-rmse:0.073394 
[36]	train-rmse:0.069277 
[37]	train-rmse:0.067055 
[38]	train-rmse:0.062119 
[39]	train-rmse:0.060

In [25]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.256603 
[2]	train-mlogloss:1.014248 
[3]	train-mlogloss:0.842610 
[4]	train-mlogloss:0.705493 
[5]	train-mlogloss:0.599258 
[6]	train-mlogloss:0.518946 
[7]	train-mlogloss:0.453682 
[8]	train-mlogloss:0.402126 
[9]	train-mlogloss:0.354561 
[10]	train-mlogloss:0.311876 
[11]	train-mlogloss:0.279108 
[12]	train-mlogloss:0.252188 
[13]	train-mlogloss:0.225059 
[14]	train-mlogloss:0.205806 
[15]	train-mlogloss:0.182341 
[16]	train-mlogloss:0.165101 
[17]	train-mlogloss:0.149081 
[18]	train-mlogloss:0.136290 
[19]	train-mlogloss:0.125094 
[20]	train-mlogloss:0.115789 
[21]	train-mlogloss:0.107649 
[22]	train-mlogloss:0.099135 
[23]	train-mlogloss:0.092194 
[24]	train-mlogloss:0.086486 
[25]	train-mlogloss:0.081004 
[26]	train-mlogloss:0.076443 
[27]	train-mlogloss:0.072216 
[28]	train-mlogloss:0.068337 
[29]	train-mlogloss:0.064984 
[30]	train-mlogloss:0.061798 
[31]	train-mlogloss:0.058989 
[32]	train-mlogloss:0.056526 
[33]	train-mlogloss:0.054355 
[34]	train-mlogloss

In [ ]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.291564 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1515
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 18
[LightGBM] [Info] Start training from score -1.490518
[LightGBM] [Info] Start training from score -2.153812
[LightGBM] [Info] Start training from score -1.728929
[LightGBM] [Info] Start training from score -0.935240
[LightGBM] [Info] Start training from score -2.422076
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [39]:
#dtrain_lg_mean_reg

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  2.998900e+00  1.001274e+00  1.281730e-03  2.000154e+00  1.999984e+00
  [6]  2.999838e+00  1.757035e+00  3.000291e+00  2.000136e+00  3.981746e-04
 [11]  3.997620e+00  2.811661e+00  1.000427e+00  2.031235e-04  1.870746e+00
 [16]  3.994956e+00  3.997380e+00  3.001468e+00  2.420797e-03  2.998992e+00
 [21]  3.999556e+00  2.323490e-01  3.999452e+00  1.888453e+00 -1.737431e-04
 [26]  2.998425e+00  2.181528e+00  1.719674e+00 -6.323862e-06  1.396008e-03
 [31]  1.000329e+00  7.791129e-01  2.795498e-04 -9.638430e-02  1.728890e-03
 [36]  1.936645e+00  3.998697e+00 -9.243509e-04 -8.374673e-02  5.476219e-04
 [41]  2.236736e-03  3.389964e+00  1.938444e+00  1.803641e+00  2.997916e+00
 [46] -3.596561e-04  2.124444e+00  2.488915e+00  1.999904e+00  1.998930e+00
 [51]  5.225741e-03  2.998248e+00  4.000343e+00  2.000595e+00 -1.343775e-03
 [56]  1.999175e+00  1.999842e+00 -2.877775e-04  2.000294e+00  3.931866e-03
 [61]  2.999005e+00  1.999196e+00  3.003875e+00  2.002320e+00  3.999667e+00
 [66]  3.998870e+00  2.999927e+00  1.999528e+00 -8.118576e-04  3.999239e+00
 [71]  8.151279e-04  9.997630e-01  3.999865e+00  1.235725e+00  2.178630e-03
 [76]  2.998831e+00  1.755082e+00  3.685568e+00  2.366603e+00 -2.489236e-03
 [81]  1.000967e+00  3.999047e+00  2.270919e+00  2.101832e+00  1.001098e+00
 [86]  4.000434e+00  2.345570e+00  2.001816e+00  1.999868e+00  1.506207e+00
 [91]  2.971317e+00  4.000038e+00 -1.174226e-03  1.054999e-03 -1.127782e-02
 [96]  2.000170e+00  2.000313e+00  2.000113e+00  2.999795e+00  2.778827e+00
[101]  4.000269e+00  1.145054e-04 -1.824999e-05  2.999135e+00  2.998502e+00
[106]  2.999593e+00  1.301302e+00  3.281950e-01 -7.745499e-04  8.149420e-04
[111]  2.513357e+00  3.275501e+00  3.999214e+00  2.000128e+00  1.913719e+00
[116]  3.000541e+00  1.868198e+00  2.617981e+00  1.098256e-03  2.065264e+00
[121]  2.000453e+00  1.999326e+00  1.998956e+00  3.999963e+00  3.999901e+00
[126]  4.000098e+00  2.841481e+00  3.735084e+00  3.999247e+00 -9.025181e-05
[131]  1.000339e+00  3.949651e+00  3.527112e+00  1.000209e+00  3.999753e+00
[136]  3.999425e+00  3.002582e+00  3.002825e+00  4.000527e+00  4.000928e+00
[141]  4.000114e+00  2.004932e+00  3.999125e+00  2.844660e+00  3.998060e+00
[146]  3.072301e+00  2.731689e+00  3.996756e+00  1.000680e+00  2.999666e+00
[151] -5.493627e-05  2.506678e-01  3.999406e+00  3.000833e+00  1.000119e+00
[156]  3.293974e+00  2.966321e+00  3.259138e+00  3.999836e+00  2.999474e+00
[161]  2.975289e+00  3.999811e+00  2.999871e+00  3.997173e+00  3.999979e+00
[166]  1.001474e+00  2.999801e+00  3.575359e+00  3.625827e+00  2.938728e+00
[171]  3.999813e+00  3.998931e+00  3.999447e+00  3.483201e+00  3.010342e+00
[176]  2.999735e+00  3.397216e+00  3.997474e+00  3.002226e+00  3.999703e+00
[181]  3.994985e+00  2.975553e+00  3.485362e+00  3.000567e+00  3.999878e+00
[186]  4.000558e+00  3.998142e+00  4.000439e+00  3.999876e+00  3.468690e+00
[191]  3.587948e+00  3.996145e+00  3.999973e+00  3.420897e+00  3.000830e+00
[196]  3.999324e+00  3.999899e+00  3.004110e+00  3.999634e+00  3.516340e+00
[201]  3.000490e+00  2.908895e+00  2.456892e+00  3.594824e+00  4.000323e+00
[206]  3.722353e+00  3.576247e+00  4.001410e+00  4.000111e+00  3.999996e+00
[211]  2.004063e+00  3.726764e+00  3.001742e+00  3.999474e+00  3.999787e+00
[216]  3.999607e+00  3.397871e+00  3.403164e+00  4.000797e+00  3.999906e+00
[221]  3.999927e+00  3.999860e+00  3.999467e+00  2.797772e+00  3.999194e+00
[226]  4.000430e+00  3.999570e+00  3.999022e+00  4.000379e+00  9.998526e-01
[231]  3.999397e+00  4.000838e+00  3.394659e+00  3.000416e+00  3.003126e+00
[236]  3.001358e+00  3.999579e+00  3.486900e+00  3.609201e+00  2.602584e+00
[241]  3.000057e+00  3.794910e+00  3.999558e+00  3.223657e+00  3.000619e+00
[246]  3.149184e+00  4.000168e+00  3.999633e+00  4.000140e+00  4.000160e+00
[251]  3.998335e+00  3.246651e+00  4.001546e+00  4.000440e+00  2.116764e+00
[256]  3.003586e+00  3.998344e+00  4.000384e+00  3.999746e+00  3.681593e+00
[261]  4.000072e+00  3.413547e+00  3.999823e+00  3

In [41]:
xgbclsmin

[1] 3 1 0 2 2 3 2 3 2 0 4 3 1 0 2 4 4 3 0 3 4 0 4 4 0 3 0 4 0 0 1 1 0 0 0 2 4
 [38] 0 0 0 0 3 3 0 3 0 3 4 2 2 0 3 4 2 0 2 2 0 2 0 3 2 4 2 4 4 3 2 0 4 0 1 4 4
 [75] 0 3 2 4 3 0 1 4 4 3 1 4 2 2 2 3 3 4 0 0 0 2 2 2 3 4 4 0 0 3 3 3 0 0 0 0 3
[112] 4 4 2 4 3 2 2 0 3 2 2 2 4 4 4 3 4 4 0 1 4 4 1 4 4 3 3 4 4 4 2 4 1 4 4 4 4
[149] 1 3 0 0 4 3 1 4 4 1 4 3 4 4 3 4 4 1 3 4 3 4 4 4 4 4 4 3 4 4 3 4 4 4 4 3 4
[186] 4 4 4 4 4 4 4 4 4 3 4 4 3 4 4 3 4 4 4 4 4 4 4 4 4 2 4 3 4 4 4 4 4 4 4 4 4
[223] 4 4 4 4 4 4 4 1 4 4 4 3 3 3 4 4 4 4 3 4 4 4 3 4 4 4 4 4 4 4 4 4 1 4 4 4 4
[260] 4 4 4 4 4 4 3 4 1 4 3 4 4 3 3 3 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 2 4 4 3 2 2
[297] 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 2 4 4 4 4 4 4 3 4 4 4 4 4 4 4 3 3 4 1 4
[334] 4 4 0 4 0 1 3 1 0 4 3 4 4 4 4 4 3 2 3 4 3 4 4 4 4 0 3 4 0 1 3 4 4 3 3 1 1
[371] 0 3 0 4 4 1 0 2 0 1 1 3 4 4 3 3 0 0 3 0 2 1 0 4 4 3 0 4 4 4 0 0 4 4 1 2 4
[408] 1 1 1 1 2 4 4

In [42]:
lgbregmin

[1]  1.98399617  1.23960111  0.24022451  2.20747568  1.75506529  2.49350499
  [7]  1.41712373  2.63913903  1.80115384  0.29937208  3.96003243  2.36331486
 [13]  1.39313728  0.46762615  1.35979889  3.83309052  3.21806596  3.41101705
 [19]  0.35873135  2.65886062  3.95048956  1.11237270  3.84139064  2.15710879
 [25]  0.30791274  2.74057369  1.94363478  1.35503649  0.10563399  0.49120917
 [31]  1.06779039  0.87878717 -0.06928045  0.80113946  0.87281925  2.26826509
 [37]  3.07619245  0.15675357  0.50796319  0.40739756  0.47549903  3.17336843
 [43]  1.95789267  1.15903180  2.07208832  1.04859416  1.98573442  1.95891042
 [49]  2.38314162  1.87067105  0.72303592  2.31636520  3.53912966  2.22760873
 [55] -0.07549419  1.90845708  2.02888992  0.36573314  2.01384258  0.74152170
 [61]  2.34435491  1.66949965  2.40230541  2.08587041  3.72885766  3.50636441
 [67]  2.73225234  1.88366087  0.31248111  3.68488728  0.39161018  1.45638566
 [73]  2.98096719  2.10408721  0.98625593  2.98909654  2.74139168  3.04446749
 [79]  2.98066747  0.15091116  1.19593739  3.82646954  1.92019563  1.76521396
 [85]  1.21831027  3.77474886  2.29278796  2.61431059  1.91990998  2.68384899
 [91]  3.21852773  3.98601508  0.36016649  0.68971898  0.67287566  2.02432242
 [97]  1.98211589  2.34534434  2.92132735  3.41076054  3.74606587  0.47054442
[103]  0.25739643  2.83780109  2.83720726  2.85090037  1.69121187  0.99234493
[109]  0.45330000  0.57930971  2.41435059  2.50359923  3.07525709  1.81687287
[115]  2.57913809  2.17548973  1.94206357  3.35156004  0.04116389  1.61009838
[121]  2.18406865  2.06769786  1.76696919  4.03252290  4.07943976  3.95795885
[127]  3.74639897  3.27490391  3.91439976  0.53021796  1.34484792  3.57480685
[133]  3.39214264  2.08301568  3.16130612  3.86683672  3.29506157  3.22294439
[139]  3.95387269  3.68474367  3.73180840  2.53242887  3.57256408  2.82390311
[145]  3.78290773  3.43831235  2.77693210  3.57100624  1.62719144  2.79096853
[151]  0.24279589  1.65706202  3.78879730  3.17335485  0.87425871  2.92458140
[157]  1.81359132  4.03043662  3.61761312  2.66203951  4.13115026  3.97550938
[163]  3.20789666  3.80804292  3.83945279  1.23131061  3.18286338  3.36643843
[169]  3.28721146  4.11592938  4.00724428  3.89425377  3.98658184  2.86608704
[175]  3.53108838  3.39317334  3.98900695  3.74816389  3.36299834  3.82564789
[181]  3.77315577  3.55478298  4.08328816  3.28568281  3.88670329  3.89603352
[187]  3.84406228  4.08020144  3.87912106  3.57880046  3.92324651  3.63329511
[193]  4.09665890  4.12703643  3.26674692  3.93302783  4.03315687  3.48596414
[199]  3.94366472  3.69025089  3.30291771  2.75922700  3.05824506  4.01384833
[205]  3.84697558  3.99966683  3.93262622  3.90874800  3.95464896  3.81451565
[211]  2.45487047  4.06207398  3.39049180  3.78235345  3.84536613  3.86097386
[217]  3.86644360  4.11892998  3.90267460  3.90179951  3.85282075  3.97569060
[223]  3.65127503  3.46243405  3.77989305  4.04574546  3.63119776  3.78320308
[229]  3.93038958  1.81075732  3.77664547  3.90273486  3.83982314  3.10962796
[235]  3.17893993  3.29215624  4.12204152  4.29413124  4.05865653  3.64769884
[241]  3.26617338  4.34050209  4.08413249  3.22648247  3.07280192  3.57594677
[247]  4.21962496  3.59835614  3.85622117  3.86059254  3.50703949  4.03437027
[253]  3.75632462  3.73450874  2.46188859  3.65822446  4.06398875  3.88512436
[259]  3.89032369  3.95353556  3.92216956  3.84838451  3.93571393  3.61498526
[265]  3.73860119  3.29621168  3.74659646  1.81084859  3.76390037  3.72788870
[271]  3.98494123  3.82281457  3.48192564  3.16894760  3.10651352  3.58835161
[277]  3.94199244  3.88411695  3.68078256  3.10305902  3.75996058  3.88136854
[283]  3.94077690  4.05429300  4.00863697  3.78087364  3.87884322  3.70901127
[289]  3.63528509  3.84183238  2.14782368  3.94376822  3.96758025  3.12184152
[295]  2.50779305  2.61719700  3.89859252  3.77159625  4.16595506  3.98192345
[301]  3.96817857  3.86876690  3.50737986  3.83826184  3.96621184  3.99446305
[307]  4.01451082  3.144

In [43]:
lgbclsmin

0.0198684488,7.482524e-04,1.180506e-04,0.9737861176,0.0054791306
0.0068133942,9.874226e-01,4.857688e-04,0.0047894408,0.0004888076
0.9977334039,1.133278e-04,1.404203e-04,0.0005849914,0.0014278566
0.0009715427,4.694705e-05,9.935766e-01,0.0022733255,0.0031315387
0.0025861512,2.642407e-04,9.937149e-01,0.0015379547,0.0018967272
0.0029724965,1.221398e-03,8.248544e-04,0.9881420883,0.0068391634
0.0519431573,3.539315e-03,9.357198e-01,0.0011404978,0.0076571873
0.0055734028,3.870330e-04,3.248725e-03,0.9815684394,0.0092224001
0.0024524148,2.211538e-03,9.937093e-01,0.0011150497,0.0005117450
0.9906629789,2.507964e-05,2.271997e-03,0.0022038330,0.0048361117
0.0022169249,2.035389e-03,1.909633e-03,0.0028522676,0.9909857855


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
lgbclsminr

2
4
3
3
4
3
2
4
0
0
0


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
3,2.99890041,2,1.9839962
1,1.00127423,4,1.2396011
0,0.00128173,3,0.2402245
2,2.00015378,3,2.2074757
2,1.99998403,4,1.7550653
3,2.99983835,3,2.4935050


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
2,1.9947434664,2,1.32786123
1,0.9981291890,4,0.98953746
0,0.0009874788,3,-0.01011042
2,1.9991097450,3,1.83032697
2,1.9998356104,1,1.90943530
3,2.9969830513,3,2.44720200


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
2,1.9947434664,2,1.32786123
1,0.9981291890,4,0.98953746
0,0.0009874788,3,-0.01011042
2,1.9991097450,3,1.83032697
2,1.9998356104,1,1.90943530
3,2.9969830513,3,2.44720200


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [55]:
save(result_list, file = "Hourly_thetaf_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
0.6218309,0.6218309,0.6218309
4.9363640,4.9363640,4.9363640
6.1108991,6.1108991,6.1108991
0.9395616,0.9395616,0.9395616


In [37]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]



ERROR: Error in Ops.data.frame(importance_matrix1[, 2:4] + importance_matrix2[, : ‘+’ only defined for equally-sized data frames


In [42]:
new_row <- data.frame(Feature = 'length', Gain = 0, Cover=0, Frequency=0)
# 将数据框分割为两部分
importance_matrixf <- importance_matrix4[1:8, ]
importance_matrixs <- importance_matrix4[9:nrow(importance_matrix4), ]

# 在这两部分之间插入新的一行
importance_matrix4new <- rbind(importance_matrixf, new_row, importance_matrixs)

In [43]:
importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4new[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'h_thetaf_imp_mean.csv')

In [44]:
importance_matrix_mean

Gain,Cover,Frequency
<dbl>,<dbl>,<dbl>
0.071946398,0.098300033,0.094063377
0.036178781,0.041973100,0.045885031
0.063482393,0.057621471,0.048152865
0.056021953,0.058520100,0.060253768
0.077531353,0.095789756,0.082552070
0.040128414,0.050708703,0.051427551
0.045665102,0.056818466,0.053764806
0.041102476,0.056268047,0.052995444
0.005202328,0.004182776,0.004747209


In [31]:
importance_matrix1

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.068934073,0.095278804,0.092792793
diff1_acf1,0.049272912,0.053928031,0.055855856
diff1_acf10,0.052438811,0.042726651,0.043843844
diff2_acf1,0.046433219,0.053497531,0.066366366
diff2_acf10,0.065398864,0.100762855,0.097597598
e_acf1,0.048232369,0.062787890,0.059159159
e_acf10,0.067788437,0.063413525,0.062462462
entropy,0.062566611,0.056333554,0.045345345
length,0.006037616,0.008196373,0.009009009


In [32]:
importance_matrix2

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.090066263,0.1439188756,0.102165917
diff1_acf1,0.031869461,0.0262803823,0.034736412
diff1_acf10,0.079645349,0.0762550455,0.043318349
diff2_acf1,0.057395539,0.0506910832,0.036371067
diff2_acf10,0.093341034,0.1219195396,0.073968124
e_acf1,0.031826085,0.0613616640,0.062116878
e_acf10,0.016097341,0.0555604093,0.049039640
entropy,0.015276538,0.0669357610,0.047813649
length,0.003775123,0.0006523499,0.002043318


In [33]:
importance_matrix3

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.07457732,0.090123439,0.106554968
diff1_acf1,0.04195035,0.050157186,0.046590241
diff1_acf10,0.06256514,0.053880772,0.052469136
diff2_acf1,0.03957925,0.062525377,0.060699588
diff2_acf10,0.08981211,0.086888550,0.082010582
e_acf1,0.05170091,0.060915835,0.061728395
e_acf10,0.04976870,0.055425884,0.052469136
entropy,0.04687963,0.045746747,0.050705467
length,0.01099657,0.007882382,0.007936508


In [34]:
importance_matrix4

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.05420794,0.06387901,0.07473983
diff1_acf1,0.02162240,0.03752680,0.04635762
diff1_acf10,0.05928027,0.05762342,0.05298013
diff2_acf1,0.08067981,0.06736641,0.07757805
diff2_acf10,0.06157340,0.07358808,0.07663198
e_acf1,0.02875429,0.01776942,0.02270577
e_acf10,0.04900593,0.05287405,0.05108798
entropy,0.03968712,0.05605612,0.06811731
linearity,0.08446182,0.08748338,0.09366131


In [40]:
lgb.importance(lgb_mean_reg)

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
seas_acf1,0.29706459,0.09179173,0.07947020
linearity,0.08446182,0.08748338,0.09366131
diff2_acf1,0.08067981,0.06736641,0.07757805
diff2_acf10,0.06157340,0.07358808,0.07663198
diff1_acf10,0.05928027,0.05762342,0.05298013
curvature,0.05420794,0.06387901,0.07473983
spike,0.04942970,0.05944175,0.05676443
e_acf10,0.04900593,0.05287405,0.05108798
trend,0.04735028,0.07061633,0.06906339
